In [1]:
import numpy as np
import astropy.units as u 
import scipy.stats as st
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
%config InlineBackend.figure_format = 'svg'

# Definitions & Constants

In [3]:
def chi(o,e,δ):
    𝛘  = np.sum(((o-e)/δ)**2)
    return 𝛘

In [4]:
β      = (np.pi/180.)     # Degrees to Radians conversion.

In [5]:
def grating_spacing(m,λ,θ):
    d  = (m*λ)/(np.sin(θ*β))
    return d

In [6]:
def wavelength(m,dw,θ):
    λ  = (dw*np.sin(θ*β))/m
    return λ

In [7]:
def sigma_alpha(Δ):
    δα = (((1/(60))*β)**2 + (Δ*β)**2)**(1/2)
    return δα

In [8]:
def sigma_theta(Δ):
    δθ = sigma_alpha(Δ)/(2**(1/2))
    return δθ

In [9]:
def sigma_d(m,λ,θ,δθ):
    δd = np.abs(-(m*λ*np.cos(θ*β))/(np.sin(θ*β)**2)*(δθ))
    return δd

In [10]:
def sigma_lambda(m,dw,δdw,θ,δθ):
    δλ = ((np.sin(θ*β)*δdw/m)**2 + (dw*np.cos(θ*β)*δθ/m)**2)**(1/2)
    return δλ

In [11]:
def sigma_rydberg(λ,δλ,energy_states):
    δr = δλ*(λ**2*((1/np.full((8),2)**(2)) - (1/energy_states**(2))))**-1
    return δr

# Helium Experiment

In [12]:
"""HeData.png"""                                                # Total of 8 entries.
m          = np.array([1,1,1,1,1,1,2,2])
α_1        = np.array([11.5,10,8.78,7.72,7.67,6.5,20.25,17.25])
α_2        = np.array([11.5,10,9.2,8.5,8.2,7.22,20.33,17.42])
λ          = np.array([668,588,501,492,447,388,588,501])        # Wavelength of photon, nanometer scale.

color = ['red','yellow','green','cyan','blue','violet','yellow','green']

In [13]:
θ       = (α_1+α_2)/2
Δ       = (α_1-α_2)/2      # Below 10 arcminutes, ideally.
δθ      = sigma_theta(Δ)

d       = grating_spacing(m,λ,θ)
δd      = sigma_d(m,λ,θ,δθ)

# We obtain the grating spacing from Helium's values!
w       = δd**-2
dw      = np.sum(w*d)/np.sum(w)
δdw     = np.sum(w)**(-1/2)

In [14]:
np.mean(d),δd

(3333.869320419858,
 array([  3.38742486,   3.95002136,  52.68750136, 117.90411836,
         76.12450999, 120.09210731,   4.90550024,  11.51708972]))

In [15]:
dw,δdw

(3370.4612212962534, 2.2304470785483113)

In [16]:
data = {'$m$':m,'$\lambda [nm]$':λ,'$θ$':θ,'$d[nm]$':d,'$δ_d$':δd}
table = pd.DataFrame(data)

import seaborn as sns
cm = sns.light_palette("cornflowerblue", as_cmap=True)
s = table.style\
  .set_precision(2)\
  .format({'$m$': "{:.0f}", '$\lambda [nm]$': "{:.1f}",
                     '$θ$':"{:.2f}$^\circ$",'$d[nm]$':"{:.3g}",'$δ_d$':"{:.3g}"})\
  .background_gradient(cmap=cm,axis=0, subset='$δ_d$')

s

,$m$,$\lambda [nm]$,$θ$,$d[nm]$,$δ_d$
0,1,668.0,11.50$^\circ$,3.35e+03,3.39
1,1,588.0,10.00$^\circ$,3.39e+03,3.95
2,1,501.0,8.99$^\circ$,3.21e+03,52.7
3,1,492.0,8.11$^\circ$,3.49e+03,118
4,1,447.0,7.93$^\circ$,3.24e+03,76.1
5,1,388.0,6.86$^\circ$,3.25e+03,120
6,2,588.0,20.29$^\circ$,3.39e+03,4.91
7,2,501.0,17.34$^\circ$,3.36e+03,11.5


In [18]:
data = {'$d_w[nm]$':[dw],'$δ_{d_w}$':[δdw]}
table = pd.DataFrame(data)
s=table\
  .rename(index={0:'Weighted Average'})\
  .style.format({'$d_w[nm]$':"{:.3g}",'$δ_{d_w}$':"{:.3g}"})
s


,$d_w[nm]$,$δ_{d_w}$
Weighted Average,3.37e+03,2.23


# Hydrogen Experiment

In [ ]:
"""HData.png"""                                         # Total of 8 entries.
m     = np.array([3,3,2,2,2,1,1,1])                     # m, integer energy level.
α_1   = np.array([36,25.67,23,16.67,15,11.25,8.33,7.5]) # alpha_1, first angle in degrees.
α_2   = np.array([36,25.5,22.75,16.67,14.77,11,8,7.33]) # alpha_2, second angle in degrees.

color = ['red','cyan','red','cyan','violet','red','cyan','violet']

In [ ]:
θ     = (α_1+α_2)/2
Δ     = (α_1-α_2)/2                      # Below 10 arcminutes, ideally.
δθ    = sigma_theta(Δ)

λ     = wavelength(m,dw,θ)               # We obtain the wavelengths for Hydrogen!
δλ    = sigma_lambda(m,dw,δdw,θ,δθ)

In [ ]:
energy_states = np.array([3,4,3,4,5,3,4,5]) # We assign, based on known wavelengths for Balmer Series jumps.
expected      = np.array([656,486,656,486,434,656,486,434])

In [ ]:
errλ    = (λ - expected)
errλ                                        # [1,2,7] indices have lowest error values.
indices = [0,1,2,3,4,5,6,7]

In [ ]:
data = {'$λ[nm]$':λ,'$δ_λ$':δλ,'$n_i$':energy_states,'$δ_{balmer}[nm]$':np.abs(errλ)}
table = pd.DataFrame(data)
cm = sns.light_palette("hotpink", as_cmap=True)
s = table.style\
  .set_precision(2)\
  .format({'$λ[nm]$': "{:.1f}", '$δ_λ$': "{:.2f}",
                     '$n_2$':"{:.0f}",'$δ_{balmer}[nm]$':"{:.2f}"})\
  .background_gradient(cmap=cm,axis=0, subset=['$δ_λ$'])\
  .background_gradient(cmap=cm,axis=0, subset=['$δ_{balmer}[nm]$'])
s

In [ ]:
def rydberg(λ,n_2,n_1):
    RYDBERG = (λ*((1/n_2**(2))-(1/n_1**(2))))**-1
    return np.take(RYDBERG, indices)

In [ ]:
λ,δλ

In [ ]:
r       = rydberg(λ, n_2 = np.full((8),2), n_1=energy_states)
δr      = sigma_rydberg(λ,δλ,energy_states)

In [ ]:
w       = np.take(δr, indices)**-2
rw      = np.array(np.sum(w*r)/np.sum(w))
δrw     = np.array(np.sum(w)**(-1/2))

In [ ]:
"{:.5e}".format(rw*10**9)

In [ ]:
"{:.5e}".format(δrw*10**9)

In [ ]:
RYDBERG = np.array([10967758*10**-9])
chi1 = chi(rw,RYDBERG,δrw)

In [ ]:
r,δr

In [ ]:
data = {'$R_H[m^{-1}]$':r*10**9,'$δ_{R_H}$':np.take(δr,indices)*10**9}
table = pd.DataFrame(data)

cm = sns.light_palette("hotpink", as_cmap=True)
s = table.style\
  .set_precision(2)\
  .format({'$R_H[m^{-1}]$': "{:.4e}", '$δ_{R_H}$': "{:.2e}"})\
  .background_gradient(cmap=cm,axis=0, subset=['$δ_{R_H}$'])
s

In [ ]:
indices = [1,2,4,7]

r       = rydberg(λ, n_2 = np.full((8),2), n_1=energy_states)
δr      = sigma_rydberg(λ,δλ,energy_states)

w       = np.take(δr, indices)**-2
rw      = np.append(rw,(np.sum(w*r)/np.sum(w)))
δrw     = np.append(δrw,(np.sum(w)**(-1/2)))

chi2    = chi((np.sum(w*r)/np.sum(w)),RYDBERG,(np.sum(w)**(-1/2)))
chi     = np.append(chi1,chi2)

In [ ]:
chi,rw,δrw

In [ ]:
data = {'$R_{H_w}[m^{-1}]$':rw*10**9,'$δ_{R_{H_w}}$':δrw*10**9,'$\\tilde{\chi}^2$':chi}
table = pd.DataFrame(data)
s = table\
  .rename(index={0:'$[:]$ Weighted Average',1:'$[1,2,4,7]$ Weighted Average'})\
  .style.format({'$R_{H_w}[m^{-1}]$':"{:.4e}",'$δ_{R_{H_w}}$':"{:.2e}",'$\\tilde{\chi}^2$':"{:.3g}"})
s

# Helium-Neon LASER

In [ ]:
"""HeNeData.png"""                            # Total of 3 entries.
m          = np.array([1,2,3])
α_1        = np.array([10.5,22,33.5])
α_2        = np.array([11,22.5,34.17])
θ          = np.array([10.75,22.25,33.84])
λ          = np.array([628.06,637.49,624.96])

color = ['red','red','red']

In [ ]:
θ     = (α_1+α_2)/2
Δ     = (α_1-α_2)/2                      # Below 10 arcminutes, ideally.
δθ    = sigma_theta(Δ)

λ     = wavelength(m,dw,θ)
δλ    = sigma_lambda(m,dw,δdw,θ,δθ)

# We calculate the weighted average for LASER wavelength.
w       = δλ**-2
λw      = np.sum(w*λ)/np.sum(w)
δλw     = np.sum(w)**(-1/2)

In [ ]:
λ,δλ

In [ ]:
data = {'$λ[nm]$':λ,'$δ_λ$':δλ}
table = pd.DataFrame(data)

cm = sns.light_palette("red", as_cmap=True)
s = table.style\
  .format({'$λ[nm]$': "{:.4g}", '$δ_λ$': "{:.3g}"})\
  .background_gradient(cmap=cm,axis=0, subset=['$δ_λ$'])
#   .set_caption("Calculated He-Ne LASER Wavelengths")
s

In [ ]:
λw,δλw

In [ ]:
data = {'$λ_w[nm]$':[λw],'$δ_{λ_w}$':[δλw]}
table = pd.DataFrame(data)
s = table\
  .rename(index={0:'Weighted Average'})\
  .style.format({'$λ_w[nm]$':"{:.4g}",'$δ_{λ_w}$':"{:.3g}"})
#   .set_caption("Average Wavelength of He-Ne LASER")
s